In [1]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta
from itertools import chain

from datetime import date
from shapely.geometry import MultiPolygon


sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [58]:
path = "/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/Final_dataset_as_of_20240209.csv" 
fires = fv.prep_fire_files(path)

In [59]:
ciffc = pd.read_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/CIFFC_data/ciffc_all_canada.csv")
ciffc = ciffc[ciffc.field_agency_code == "qc"]

ciffc = gpd.GeoDataFrame(ciffc, geometry= gpd.points_from_xy(ciffc.field_longitude, ciffc.field_latitude), crs = "4326")
ciffc = ciffc.to_crs("3571")

In [60]:
fires.columns

Index(['fireID', 't', 'geometry', 'Unnamed: 0.1', 'csv_geometry', 'Unnamed: 0',
       'InterCloud', 'lat', 'lon', 'current_mag', 'error_elps', 'num_station',
       'num_candidates', 'num_strikes', 'num_strikes_10_days',
       'no_strikes_in_time', 'viirs_pix_count', 'n_pixels', 'n_newpixels',
       'farea', 'fperim', 'flinelen', 'duration', 'pixden', 'meanFRP',
       'csv_geometry', 'lon_centroid', 'lat_centroid', 'GEOS-5.IMERGEARLY',
       'FWI', 'FWI_lead_1', 'FWI_lead_2', 'FWI_lead_3', 'FWI_lead_4',
       'FWI_lead_5', 'FWI_lead_6', 'FWI_lead_7', 'FWI_lead_8', 'pre_fire'],
      dtype='object')

In [61]:
fires = fires[['fireID', 't', 'geometry', 
       'InterCloud', 'lat', 'lon', 'current_mag', 'error_elps', 'num_station',
       'num_candidates', 'num_strikes', 'num_strikes_10_days','n_pixels', 'n_newpixels',
       'farea', 'fperim', 'flinelen', 'duration', 'pixden', 'meanFRP','lon_centroid', 'lat_centroid']]

In [62]:
fires.columns

Index(['fireID', 't', 'geometry', 'InterCloud', 'lat', 'lon', 'current_mag',
       'error_elps', 'num_station', 'num_candidates', 'num_strikes',
       'num_strikes_10_days', 'n_pixels', 'n_newpixels', 'farea', 'fperim',
       'flinelen', 'duration', 'pixden', 'meanFRP', 'lon_centroid',
       'lat_centroid'],
      dtype='object')

In [64]:
fires = fires.drop_duplicates(subset= ['geometry', 'InterCloud', 'lat', 'lon', 'current_mag',
       'error_elps', 'num_station', 'num_candidates', 'num_strikes',
       'num_strikes_10_days', 
       'n_pixels', 'n_newpixels', 'farea', 'fperim', 'flinelen', 'duration',
       'pixden', 'meanFRP', 'lon_centroid', 'lat_centroid'])
fires = fires.sort_values(by = "t")

In [66]:
fires.loc[fires.fireID == "9791", [ 'fireID', 't', 
       'InterCloud', 
       'num_candidates', 'num_strikes', 'num_strikes_10_days', 'farea', 'n_pixels']] ## NOte, I am seeing viirs_pix_counts that onlu states long after the algorithm started tracking the fire. Merge issue coming up here? Will delete

,fireID,t,InterCloud,num_candidates,num_strikes,num_strikes_10_days,farea,n_pixels
0,9791,2023-06-10 15:24:18.066,0.0,2.0,2.0,50224.0,NaN,NaN
125,9791,2023-06-11 00:00:00.000,NaN,NaN,NaN,NaN,0.220539,2.0
16,9791,2023-06-11 07:07:00.000,NaN,NaN,NaN,NaN,NaN,NaN
127,9791,2023-06-11 12:00:00.000,NaN,NaN,NaN,NaN,1.181963,13.0
126,9791,2023-06-12 00:00:00.000,NaN,NaN,NaN,NaN,1.181963,13.0
124,9791,2023-06-12 12:00:00.000,NaN,NaN,NaN,NaN,3.888546,30.0
128,9791,2023-06-13 00:00:00.000,NaN,NaN,NaN,NaN,15.621517,107.0
121,9791,2023-06-13 12:00:00.000,NaN,NaN,NaN,NaN,90.350962,543.0
118,9791,2023-06-14 00:00:00.000,NaN,NaN,NaN,NaN,90.350962,543.0
117,9791,2023-06-16 12:00:00.000,NaN,NaN,NaN,NaN,96.846414,562.0


In [67]:
### Renaming columns to be more intutitive

fires = fires.rename(columns = {"num_candidates": 'num_candidate_ig_strike', 
                       'num_strikes' : "num_strikes_at_t_of_candidates", 
                        'num_strikes_10_days':'num_strikes_in_region_during_window', 
                       'duration': 'fire_duration', 
                       'lat': 'lt_lat', 
                       'lon': 'lt_lon', 
                       'lon_centroid': 'fr_lon_centroid', 
                       'lat_centroid': 'fr_lat_centroid'})

#8614

In [81]:
fires = fires.sjoin(ciffc)

In [85]:
fires = fires[['fireID', 't', 'geometry', 'InterCloud', 'lt_lat', 'lt_lon',
       'current_mag', 'error_elps', 'num_station', 'num_candidate_ig_strike',
       'num_strikes_at_t_of_candidates', 'num_strikes_in_region_during_window',
       'n_pixels', 'n_newpixels', 'farea', 'fperim', 'flinelen',
       'fire_duration', 'pixden', 'meanFRP', 'fr_lon_centroid',
       'fr_lat_centroid',
       'field_situation_report_date', 
       'field_system_fire_cause']]

In [86]:
test.columns

Index(['fireID', 't', 'geometry', 'InterCloud', 'lt_lat', 'lt_lon',
       'current_mag', 'error_elps', 'num_station', 'num_candidate_ig_strike',
       'num_strikes_at_t_of_candidates', 'num_strikes_in_region_during_window',
       'n_pixels', 'n_newpixels', 'farea', 'fperim', 'flinelen',
       'fire_duration', 'pixden', 'meanFRP', 'fr_lon_centroid',
       'fr_lat_centroid', 'index_right', 'field_agency_fire_id',
       'field_agency_code', 'field_agency_data_timezone',
       'field_situation_report_date', 'field_status_date',
       'field_stage_of_control_status', 'field_system_fire_cause',
       'field_response_type', 'field_fire_size', 'field_latitude',
       'field_longitude', 'full_count'],
      dtype='object')

In [87]:
fires.to_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/Whole_CA_lightning_Anlysis/Data/Quebec_20_day_lightning_file.csv")

In [84]:
fires

,fireID,t,geometry,InterCloud,lt_lat,lt_lon,current_mag,error_elps,num_station,num_candidate_ig_strike,...,farea,fperim,flinelen,fire_duration,pixden,meanFRP,fr_lon_centroid,fr_lat_centroid,field_situation_report_date,field_system_fire_cause
52848,8244,2023-05-31 12:00:00.000,"MULTIPOLYGON (((3930737.272 1760069.104, 39307...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,107.589803,66.929460,52.219824,2.0,5.790512,57.920832,-65.946038,50.579925,2023-05-28T15:30:00.000Z,H
52847,8244,2023-06-01 00:00:00.000,"POLYGON ((3930746.385 1760084.029, 3930756.849...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,131.870902,71.169991,13.261920,2.5,5.823878,6.439586,-65.965612,50.574973,2023-05-28T15:30:00.000Z,H
52846,8244,2023-06-01 12:00:00.000,"POLYGON ((3930756.849 1760098.040, 3930768.573...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,160.697984,84.711554,20.804594,3.0,5.905488,34.427127,-65.983709,50.562016,2023-05-28T15:30:00.000Z,H
52843,8244,2023-06-02 00:00:00.000,"MULTIPOLYGON (((3931717.481 1761319.538, 39318...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,214.820663,128.756444,46.180681,3.5,5.129860,6.105359,-66.024975,50.538328,2023-05-28T15:30:00.000Z,H
52841,8244,2023-06-02 12:00:00.000,"MULTIPOLYGON (((3931880.694 1762005.363, 39315...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,215.276553,129.091685,2.201291,4.0,5.300159,12.947179,-66.025163,50.538354,2023-05-28T15:30:00.000Z,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64149,12639,2023-07-23 00:00:00.000,"MULTIPOLYGON (((4057182.916 944804.837, 405718...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,85.917733,77.890852,5.848028,15.5,7.949465,1.981290,-76.857897,51.986713,2023-07-05T16:43:00.000Z,N
64108,12639,2023-07-23 12:00:00.000,"MULTIPOLYGON (((4057184.866 944786.373, 405718...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,114.099418,90.782349,27.017215,16.0,7.432115,27.244485,-76.840902,51.986723,2023-07-05T16:43:00.000Z,N
64090,12639,2023-08-03 12:00:00.000,"MULTIPOLYGON (((4057184.866 944786.373, 405718...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,114.099418,90.782349,0.000000,16.0,7.432115,0.000000,-76.840902,51.986723,2023-07-05T16:43:00.000Z,N
100403,14729,2023-07-25 12:00:00.000,"POLYGON ((3912262.995 821068.554, 3912179.425 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.884400,10.112670,1.610507,2.5,4.708869,5.440000,-78.130346,53.551974,2023-07-22T13:00:00.000Z,N


In [18]:
def get_lt(lt_string = "Lt_CA_Boreal_"):
    files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/computed_data/" + lt_string +"*.csv")
    return(concat_subsets_lower_mem(files))

def concat_subsets_lower_mem(files):
    df = []
    for f in files:
        #manyfr = pd.read_csv(f, parse_dates=['t'], date_parser=dateparse)
        manyfr = pd.read_csv(f)
        manyfr = gpd.GeoDataFrame(manyfr)
        
        
        manyfr.t = manyfr.t.astype("datetime64[ns]")
        #manyfr.t = manyfr.t.astype("str")

        
        #manyfr.t.tz("UCT")
        df.append(manyfr)
    df = pd.concat(df)
    #df.t = df.t.astype("datetime64[ns]")
    return(df)


lt = get_lt() 

lt = gpd.GeoDataFrame(lt, geometry=gpd.points_from_xy(lt.lon, lt.lat), crs=4326) #4674
prov = fv.ca_prov()
lt = lt.sjoin(prov)
lt = lt.to_crs("3571")

lt = lt[lt.prov_name_en == 'Quebec']

lt = lt[['InterCloud', 't', 'lat', 'lon', 'current_mag',
       'multiplicity_0', 'accr', 'error_elps', 'num_station', 'geometry']]

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [33]:
year = '2023'
path_region= "BOREAL_NRT_3571_DPS" 
check_last = False 
FWI_source = "station" 
#ids = ids[0:5]

fireID = "9791"
min_days = 10


## Read in the largefire file of the fireID
fr = fv.load_large_fire(fireID, year = year, path_region= path_region, s3_path = True) ## Cluster of 2 fires. 


fr = fr.to_crs("3571")

oldest_perim = fr[fr.t == fr.t.max()]
first_perim = fr[fr.t == fr.t.min()]

# if(check_last == True):
#     oldest_perim.t = oldest_perim.t.astype("str")
#     m = oldest_perim.explore()
#     return(m)




 ## Subset lightning by time and space

## TO DO: Figure out which CA ecoregion/province the fire is in and subset lighting by that? 
#print("Not yet subseting spatially beyond quebec. Assuming quebec bounding box")

min_threshold = fr.t.astype('datetime64[ns]').min() - timedelta(days = min_days)
possible_lt = lt[lt.t <= fr.t.min()]
possible_lt = possible_lt[possible_lt.t >= min_threshold]

first_perim.geometry = first_perim.buffer(750*2) ## Two viirs pixels???
join_lt = gpd.sjoin(possible_lt, first_perim, predicate = 'within', how = "inner")
join_lt["no_strikes_in_time"] = False

if (len(possible_lt == 0)):
    join_lt["no_strikes_in_time"] = True

if (len(join_lt[join_lt.InterCloud.isna()].InterCloud) == len(join_lt.InterCloud)): ### Has strikes
    join_lt["num_candidates"] = 0
    join_lt["num_strikes"] = len(possible_lt)
    join_lt["num_strikes_10_days"] = len(possible_lt)
else:
    ## Extract "denominator" or the # of strikes from same period
    denominator = possible_lt[possible_lt.t >= join_lt.t_left.min()]
    denominator = denominator[denominator.t <= join_lt.t_left.max()]
    join_lt["num_candidates"] = len(join_lt)  ### Which stikes were both in spatial and temporal range of the fire start
    join_lt["num_strikes"] = len(denominator) ### Number of strikes between the time period of the first strike and last strike 
    join_lt["num_strikes_10_days"] = len(possible_lt) ## Number of stikes in the whole region in the time period

['F9791']


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:595: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:596: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y
/projects/myenvs/fireatlas_oct4/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [30]:
oldest_perim.t = oldest_perim.t.astype("str")
m = oldest_perim.explore()

/projects/myenvs/fireatlas_oct4/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [34]:
join_lt

,InterCloud,t_left,lat_left,lon_left,current_mag,multiplicity_0,accr,error_elps,num_station,geometry,...,duration,pixden,meanFRP,t_right,lon_right,lat_right,no_strikes_in_time,num_candidates,num_strikes,num_strikes_10_days
2344,0,2023-06-10 15:24:18.066,50.728772,-76.259208,-4973.0,0,0.0,12.0,3.0,POINT (4172039.574 1020180.982),...,0.0,9.068709,0.635,2023-06-11,-76.26627,50.727235,True,2,2,49096
2344,0,2023-06-10 15:24:18.066,50.728772,-76.259208,-4973.0,0,0.0,12.0,3.0,POINT (4172039.574 1020180.982),...,0.0,9.068709,0.635,2023-06-11,-76.26627,50.727235,True,2,2,49096


In [35]:
len(join_lt[join_lt.InterCloud.isna()].InterCloud) == len(join_lt.InterCloud)

False